In [84]:
import pandas as pd

In [85]:
nric_col = 'nric'

In [86]:
response_file = './data/form 1 (Responses).xlsx'
response_df = pd.read_excel(response_file)
response_df

,Timestamp,nric,1. question a,2. question b,3. choose 3 elders [choice 1],3. choose 3 elders [choice 2],3. choose 3 elders [choice 3]
0,2024-06-05 15:42:24.559,s1234567a,yes,no,person a,person b,person c
1,2024-06-05 15:42:38.290,s23456789b,yes,yes,person a,person c,person e
2,2024-06-05 15:43:06.433,badnric,no,no,person a,person c,person e
3,2024-06-05 15:43:24.385,s4567890d,no,yes,person b,person c,person d
4,2024-06-05 15:43:51.842,s3456789c,yes,no,person b,person d,person c


In [87]:
attendance_file = 'data/nric-attendance-masterlist.xlsx'
attendance_df = pd.read_excel(attendance_file)
attendance_df = attendance_df[[nric_col, 'attendance']].dropna(axis=0)
attendance_df

,nric,attendance
0,s1234567a,1.0
1,s2345678b,1.0
2,t0000000x,0.0
3,s3456789c,1.0
4,s4567890d,0.0


In [88]:
# do nric filtering


In [89]:
from collections import Counter

In [105]:
questions = [c for c in response_df.columns if c not in ['Timestamp', nric_col]]
questions = [[q.split('. ')[0], q] for q in questions]
question_nums = [q[0] for q in questions]
question_counts = Counter(question_nums)

single_choice_questions = [q[0] for q in question_counts.items() if q[1] == 1]
multi_choice_questions_with_counts = [q for q in question_counts.items() if q[1] != 1]

unique_questions = [[q[0], q[1].split(' [')[0]] for q in questions]
unique_questions_dict = dict()
for q_num, full_q in unique_questions:
    if q_num not in unique_questions_dict:
        unique_questions_dict[q_num] = full_q

In [107]:
print(f"The single choice questions are {single_choice_questions}")
print(f"The multi choice questions with choice counts are {multi_choice_questions_with_counts}")

The single choice questions are ['1', '2']
The multi choice questions with choice counts are [('3', 3)]


In [116]:
# handle ties

for q_num, full_q in sorted(unique_questions_dict.items(), key=lambda x: x[0]):
    print(full_q)
    print()
    if q_num in single_choice_questions:
        counts = sorted(list(response_df[[full_q]].groupby(full_q).size().items()), key=lambda x: -x[1])
        print(counts)
        print(f'\nThe majority decision is {counts[0][0]} with {counts[0][1]} votes.')
    elif q_num in [x[0] for x in multi_choice_questions_with_counts]:
        question_cols = [q[1] for q in questions if q[0] == q_num]
        all_vals = response_df[question_cols].values.flatten()
        vals_counter = sorted(Counter(all_vals).items(), key=lambda x: -x[1])
        print(vals_counter)
        num_choices = [x[1] for x in multi_choice_questions_with_counts if x[0] == q_num][0]
        results = vals_counter[:num_choices]
        results = [x[0] for x in results]
        print(f'\nThe top {num_choices} options with the most votes are {results}')
    else:
        print('ERROR')
        
    print('-' * 20)

1. question a

[('yes', 3), ('no', 2)]

The majority decision is yes with 3 votes.
--------------------
2. question b

[('no', 3), ('yes', 2)]

The majority decision is no with 3 votes.
--------------------
3. choose 3 elders

[('person c', 5), ('person a', 3), ('person b', 3), ('person e', 2), ('person d', 2)]

The top 3 options with the most votes are ['person c', 'person a', 'person b']
--------------------
